## Design Prompts Chains and Retrieval Mechanisms

Load configs for database, Azure OpenAI, and other resources as environment variables.

In [1]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "llm_pgvector.env" # change to your own .env file name
config = dotenv_values(env_name)

# Connect to Flex Postgres (PG)  for retrievals

In [2]:
import psycopg2
from psycopg2 import pool
from psycopg2 import Error

host = config["HOST"]
dbname = config["DBNAME"] 
user = config["USER"] 
password = config["PASSWORD"] 
sslmode = config["SSLMODE"] 

# Build a connection string from the variables
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20,conn_string)
if (postgreSQL_pool):
    print("Connection pool created successfully")

# Use getconn() to get a connection from the connection pool
connection = postgreSQL_pool.getconn()
cursor = connection.cursor()

Connection pool created successfully


In order to use pgvector, we need to first create the vector extension as described in this [link](https://learn.microsoft.com/en-us/azure/postgresql/flexible-server/how-to-use-pgvector) and shown below.

In [3]:
# Open a cursor to perform database operations
# This is likely in case extension isn't already created from portal.
cursor = connection.cursor()

try:
    # Start a new transaction
    cursor.execute("BEGIN;")

    # Previous transaction statements
    # ...

    # Check if the extension already exists
    extension_query = "SELECT * FROM pg_extension WHERE extname = 'vector';"
    cursor.execute(extension_query)
    extension_exists = cursor.fetchone()

    if not extension_exists:
        # Extension does not exist, create it
        create_extension_query = "CREATE EXTENSION vector;"
        cursor.execute(create_extension_query)
        connection.commit()
    else:
        # Extension already exists, pass through
        pass

    # Commit the transaction
    cursor.execute("COMMIT;")
except Exception as e:
    # An error occurred, rollback the transaction
    cursor.execute("ROLLBACK;")
    raise e
finally:
    # Close the cursor
    cursor.close()


Examine the total number of items based on filter_id1_name and Chunk Embedding

In [4]:
# Assuming you have already established a connection and have a cursor object
table_name = 'EarningsCallChunksEmbedding'
# Rollback the current transaction
connection.rollback()
cursor = connection.cursor()
# Execute the SELECT statement

try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(f"Number of items: {row}")
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")
    
try:
    cursor.execute(f"SELECT embedding FROM {table_name} limit 5;")
    rows = cursor.fetchall()
    for row in rows:
        print(f"Items ID: {row}")
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

try:
    cursor.execute(f"SELECT Ticker FROM {table_name} limit 1;")
    rows = cursor.fetchall()
    for row in rows:
        print(f"Items ID: {row}")
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

Number of items: (442,)
Items ID: ('[-0.022691457,-0.028929658,-0.019390408,-0.022535501,0.008954418,0.01016307,-0.0083241,-0.0066605792,0.006725561,-0.02278243,0.048268083,0.012456908,-0.010208556,-0.009077882,-0.018532656,-0.006952995,0.0075378264,-0.006751553,0.021703742,-0.026291419,-0.01712906,-0.007511834,-0.008837452,0.004831357,-0.028123891,-0.024225015,0.0044544656,0.0013686678,0.017207038,-0.021586776,-0.005188754,-0.024705876,-0.0062057106,-0.0039021247,-0.016791157,-0.011215766,-0.0061927144,0.01359408,0.023354266,0.0062349522,0.0060400083,-0.007583313,-0.015894417,-0.0051010293,-0.019364417,0.018051796,0.0045486884,0.005461675,0.005659868,0.03581767,0.022652468,0.012443912,-0.017168049,-0.028721718,0.009844662,0.011449698,0.005490917,0.01712906,-0.013230185,0.002316582,-0.009974624,-0.012528388,-0.01224247,0.02597951,-0.02056007,-0.020729022,-0.008265616,0.001375166,-0.0025505146,0.020638049,0.04293962,0.019923255,0.009909643,-0.011079305,0.02056007,-0.01712906,-0.03152891

Prepare for question embedding

In [5]:
import openai
import pandas as pd
import pandas as pd
import numpy as np


openai.api_type = config["OPENAI_API_TYPE"] 
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE'] 
openai.api_version = config['OPENAI_API_VERSION'] 


def createEmbeddings(text):
    response = openai.Embedding.create(input=text , engine=config["OPENAI_DEPLOYMENT_EMBEDDING"])
    embeddings = response['data'][0]['embedding']
    return embeddings

## User Asks a Question 
In this step, the code will convert the user's question to an embedding and then retieve the top K document chunks based on the users' question using the similarity. Please note that other similarity metrics can also be used.

In [6]:
userQuestion = "What was the highest earning?"
filter_col = "Ticker"
filter_val = "MSFT"
retrieve_k = 3 # for retrieving the top k reviews from the database

In [7]:
# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = createEmbeddings(userQuestion)

## Hybrid Search
In this case, we will first filter based on id range, and then do similarity search

In [8]:
from pgvector.psycopg2 import register_vector

connection = psycopg2.connect(conn_string)
# Create a cursor after the connection
# Register 'pgvector' type for the 'embedding' column
register_vector(connection)
cursor = connection.cursor()
select_query = f"SELECT Id FROM {table_name}  WHERE {filter_col} = '{filter_val}' ORDER BY Embedding <-> %s LIMIT 3"
cursor = connection.cursor()
cursor.execute(select_query, (np.array(questionEmbedding),))
results = cursor.fetchall()

In [9]:
# Use the top k ids to retrieve the actual text from the database 
top_ids = []
for i in range(len(results)):
    top_ids.append(int(results[i][0]))

print(top_ids)

[362, 194, 121]


#### Retrieve text from database

In [10]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()

format_ids = ', '.join(['%s'] * len(top_ids))

sql = f"SELECT CONCAT('PageNumber: ', PageNumber, ' ', 'Text: ', Chunk) AS concat FROM {table_name} WHERE id IN ({format_ids})"

# Execute the SELECT statement
try:
    cursor.execute(sql, top_ids)    
    top_rows = cursor.fetchall()
    for row in top_rows:
        print(row)
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")


("PageNumber: 3 Text: with our revenue growth. This in turn sets us up to participate in the secular trend where digital spend as a percentage of GDP is only going to increase. And lastly we're going to lead in the Al era, knowing that maximum enterprise value gets created during platform shifts. With that as the backdrop, the Microsoft Cloud exceeded $27 billion in quarterly revenue, up 22 percent and 29 percent in constant currency. Now, I'll highlight examples of our innovation, starting with Azure. Moving to the cloud is the best way for any customer in today's economy to mitigate demand uncertainty and energy costs, while gaining efficiencies ",)
("PageNumber: 25 Text: much higher, and on a secular basis. The question is, how many times is it, given the overall inflation adjusted economic growth? That's kind of how I look at it. Given that, I think the two things that we see, we commented on that even in the last quarter, and it's even in the outlook, which is the thing that custo

In [11]:
# create the context from the top_rows
context = ""
for row in top_rows:
    context += row[0]
    context += "\n"
    
print(context)

PageNumber: 3 Text: with our revenue growth. This in turn sets us up to participate in the secular trend where digital spend as a percentage of GDP is only going to increase. And lastly we're going to lead in the Al era, knowing that maximum enterprise value gets created during platform shifts. With that as the backdrop, the Microsoft Cloud exceeded $27 billion in quarterly revenue, up 22 percent and 29 percent in constant currency. Now, I'll highlight examples of our innovation, starting with Azure. Moving to the cloud is the best way for any customer in today's economy to mitigate demand uncertainty and energy costs, while gaining efficiencies 
PageNumber: 25 Text: much higher, and on a secular basis. The question is, how many times is it, given the overall inflation adjusted economic growth? That's kind of how I look at it. Given that, I think the two things that we see, we commented on that even in the last quarter, and it's even in the outlook, which is the thing that customers ar

# Provide answer to a user's question
We use [langchain](https://python.langchain.com/en/latest/index.html) to construct chains and add prompt template.

In [12]:
from langchain.prompts import PromptTemplate
question_prompt_template = """Use the following portion of the context document to find relevant text and answer the question in details. Extract PageNumber and LineNumber and show it in the answer. 
{context}
Question: {question}
If the answer is not found, say that answer is not available in the documentation."""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

### Define llm model

In [13]:
import os
os.environ["OPENAI_API_TYPE"] = config["OPENAI_API_TYPE"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["OPENAI_API_BASE"] = config["OPENAI_API_BASE"]
os.environ["OPENAI_API_VERSION"] = config["OPENAI_API_VERSION"]
os.environ["OPENAI_DEPLOYMENT_NAME"] = config["OPENAI_DEPLOYMENT_COMPLETION"]

In [14]:
from langchain.llms import AzureOpenAI
llm= AzureOpenAI(deployment_name=config["OPENAI_MODEL_COMPLETION"], model_name=config["OPENAI_MODEL_EMBEDDING"], temperature=0)

In [15]:
from typing import List, Optional

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader


class TextFormatter(BaseLoader):
    """Load text files."""

    def __init__(self, text: str):
        """Initialize with file path."""
        self.text = text

    def load(self) -> List[Document]:
        """Load from file path."""
        metadata = {"source": ""}
        return [Document(page_content=self.text, metadata=metadata)]

In [16]:
from langchain.chains.question_answering import load_qa_chain

loader = TextFormatter(context)
# qa_document_chain.run(input_document=context, question=userQuestion)
chain = load_qa_chain(llm, chain_type="stuff", prompt=QUESTION_PROMPT)
ans = chain({"input_documents": loader.load(), "question": userQuestion}, return_only_outputs=True)

In [17]:
ans['output_text']

'\nAnswer: Not available in the documentation.'

In [18]:
import os
import pandas as pd

# Get the current working directory
current_directory = os.getcwd()

# Navigate to the directory containing the CSV file (one level above the current directory)
data_directory = os.path.abspath(os.path.join(current_directory, '..', 'ValidationSetOfQA'))

# Construct the file path for your CSV file in the data_directory
csv_file_path = os.path.join(data_directory, 'QnAValidationSet.csv')

# Load the CSV file using pandas
df = pd.read_csv(csv_file_path)


In [19]:
df.head(2)

,Ticker,Quarter,Question,Answer,PageNumber
0,MSFT,1,How did the first quarter financial results co...,"In the first quarter, our revenue reached $50....",18
1,MSFT,1,What were the key highlights and growth trends...,"Within the commercial business segment, we wit...",19


In [20]:
filter1_name = 'Ticker'
filter2_name = 'Quarter'


In [21]:
df.columns = ['Ticker', 'Quarter', 'Question', 'Answer', 'PageNumber']
df.head(10)

,Ticker,Quarter,Question,Answer,PageNumber
0,MSFT,1,How did the first quarter financial results co...,"In the first quarter, our revenue reached $50....",18
1,MSFT,1,What were the key highlights and growth trends...,"Within the commercial business segment, we wit...",19


In [22]:
df_eval = df.copy()
#df_eval.dropna(subset=["question"] ,inplace=True)
#df_eval.reset_index(drop=True, inplace=True)
df_eval.head(3)

,Ticker,Quarter,Question,Answer,PageNumber
0,MSFT,1,How did the first quarter financial results co...,"In the first quarter, our revenue reached $50....",18
1,MSFT,1,What were the key highlights and growth trends...,"Within the commercial business segment, we wit...",19


In [23]:
values = [item for pair in zip(list(df_eval['Question']), list(df_eval['Answer'])) for item in pair]
keys = [str(i//2)+'a' if i%2==0 else str(i//2+1)+'q' for i in range(1,len(values)+2)]

userQuestions = {keys[i]:values[i] for i in range(len(keys)-1)}
filter1_vals = [item for item in list(df_eval[filter1_name]) for _ in range(2)]
filter2_vals = [item for item in list(df_eval[filter2_name]) for _ in range(2)]

In [24]:
userQuestions

{'1q': 'How did the first quarter financial results compare to the previous year, both in terms of revenue and earnings per share, and what adjustments were made for constant currency?',
 '1a': "In the first quarter, our revenue reached $50.1 billion, representing an 11 percent increase or 16 percent growth when adjusted for constant currency. Earnings per share stood at $2.35, showing a 4 percent increase or 11 percent growth in constant currency, after accounting for the net tax benefit from the previous year's first quarter.",
 '2q': 'What were the key highlights and growth trends observed in the commercial business segment during the first quarter, especially in terms of Microsoft Cloud offerings, share gains, and commercial bookings?',
 '2a': 'Within the commercial business segment, we witnessed strong demand for our Microsoft Cloud offerings, resulting in a remarkable 31 percent growth in constant currency. Moreover, we achieved share gains across multiple businesses. While comme

In [25]:
filter1_vals

['MSFT', 'MSFT', 'MSFT', 'MSFT']

In [26]:
# GPT based question answering with type checking
from langchain import PromptTemplate
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

Using a retrieve_k_chunk function to facilitate multiple query evaluations

In [27]:
def retrieve_k_chunk(retrieve_k, questionEmbedding,filter1_val, filter2_val):
    connection = psycopg2.connect(conn_string)
# Create a cursor after the connection
# Register 'pgvector' type for the 'embedding' column
    register_vector(connection)
    cursor = connection.cursor()
    print("filter1_name:", filter1_name)
    select_query = f"SELECT Id FROM {table_name} where {filter1_name} = '{filter1_val}' and {filter2_name}='{filter2_val}' ORDER BY embedding <-> %s LIMIT {retrieve_k}"
    cursor = connection.cursor()
    cursor.execute(select_query, (np.array(questionEmbedding),))
    results = cursor.fetchall()
    top_ids = []
    for i in range(len(results)):
        top_ids.append(int(results[i][0]))

    # Rollback the current transaction
    connection.rollback()

    format_ids = ', '.join(['%s'] * len(top_ids))

    sql = f"SELECT CONCAT('PageNumber: ', PageNumber, ' ', 'LineNumber: ', LineNumber, ' ', 'Text: ', Chunk) AS concat FROM {table_name} WHERE id IN ({format_ids})"

    # Execute the SELECT statement
    try:
        cursor.execute(sql, top_ids)    
        top_rows = cursor.fetchall()
    except (Exception, Error) as e:
        print(f"Error executing SELECT statement: {e}")
    finally:
        cursor.close()
    return top_rows

In [28]:
userQuestions.items()

dict_items([('1q', 'How did the first quarter financial results compare to the previous year, both in terms of revenue and earnings per share, and what adjustments were made for constant currency?'), ('1a', "In the first quarter, our revenue reached $50.1 billion, representing an 11 percent increase or 16 percent growth when adjusted for constant currency. Earnings per share stood at $2.35, showing a 4 percent increase or 11 percent growth in constant currency, after accounting for the net tax benefit from the previous year's first quarter."), ('2q', 'What were the key highlights and growth trends observed in the commercial business segment during the first quarter, especially in terms of Microsoft Cloud offerings, share gains, and commercial bookings?'), ('2a', 'Within the commercial business segment, we witnessed strong demand for our Microsoft Cloud offerings, resulting in a remarkable 31 percent growth in constant currency. Moreover, we achieved share gains across multiple business

In [29]:
def get_user_questions_answers():
    """
    Collection of user questions with known answers.
    """

    Q = []
    A = []
    Agpt = []
    i = 0
    for key, value in userQuestions.items():
        if "q" in key:
            Q.append(value)
            questionEmbedding = createEmbeddings(value)
            output = retrieve_k_chunk(retrieve_k, questionEmbedding,filter1_vals[i], filter2_vals[i])
            # create the context from the top_rows
            context = ""
            for row in top_rows:
                context += row[0]
                context += "\n"
            loader = TextFormatter(context)
            chain = load_qa_chain(llm, chain_type="stuff", prompt=QUESTION_PROMPT)
            ans = chain({"input_documents": loader.load(), "question": value}, return_only_outputs=True)
            Agpt.append(ans['output_text'])
            print(ans['output_text'])
            i+=2
        else:
            A.append(value)

    return  Q, A, Agpt

In [30]:
Q, A, Agpt = get_user_questions_answers()

filter1_name: Ticker

Answer: Not available in the documentation.
filter1_name: Ticker


Answer: The Microsoft Cloud exceeded $27 billion in quarterly revenue, up 22 percent and 29 percent in constant currency (Page 3, Line 1). Growth of new business continued to be moderated for products sold outside the Microsoft 365 suite including standalone Office 365, EMS, and Windows Commercial products (Page 14, Line 8). Microsoft saw share gains again this quarter in Azure, Dynamics, Security, and Edge (Page 14, Line 9). Microsoft 365 E5 saw healthy renewal strength and upsell and attach motions (Page 14, Line 10).


In [31]:
def get_cosine_similarities(QA_results):
    # compare cosine similarity between two vectors
    cosine_similarities = []
    for i in range(len(QA_results[0])):
        emd1 = createEmbeddings(QA_results[0][i])
        emd2 = createEmbeddings(QA_results[1][i])
        cosine_similarity_val = cosine_similarity(
            np.array(emd1).reshape(1, -1), np.array(emd2).reshape(1, -1)
        )[0][0]
        cosine_similarities.append(np.round(cosine_similarity_val, 2))
    return cosine_similarities

In [32]:
QAres = [A, Agpt]
scores = get_cosine_similarities(QAres)

In [33]:
scores

[0.68, 0.93]

For elaborate experimentation and additional evaluations for page number, see Notebook Step_5_mlflow_experimentation.ipynb.